In [1]:
import pandas as pd
import numpy as np
from sqlalchemy import create_engine
from sqlalchemy import text
from dotenv import load_dotenv
load_dotenv()
import os
import re
import json
import requests



## Load the data

In [2]:
train_files = [
    "./final/2024-05-16/2024-05-16_10.json",
    "./final/2024-05-16/2024-05-16_11.json",
]

test_files = [
    "./final/2024-05-16/teste-2024-05-16_12.json",
]

answer_files = [
    "./validation/2024-05-16/resposta-2024-05-15_08.json"
]
train1 = pd.read_json(train_files[0], encoding='latin-1')
train2 = pd.read_json(train_files[1], encoding='latin-1')

train = pd.concat([train1, train2])
test = pd.read_json(test_files[0], encoding='latin-1')

## Filter data

In [3]:
train['latitude'] = train['latitude'].str.replace(',', '.').astype(float)
train['longitude'] = train['longitude'].str.replace(',', '.').astype(float)
train['linha'] = train['linha'].astype(str)

valid_linhas = [
    '483', '864', '639', '3', '309', '774', '629', '371', '397', '100', '838', 
    '315', '624', '388', '918', '665', '328', '497', '878', '355', '138', '606', 
    '457', '550', '803', '917', '638', '2336', '399', '298', '867', '553', '565', 
    '422', '756', '186012003', '292', '554', '634', '232', '415', '2803', '324', 
    '852', '557', '759', '343', '779', '905', '108'
]

df_train = train[train['linha'].isin(valid_linhas)]

## Create dataframe with last 2 points of each line and bus order

In [4]:
df_last_two = df_train.groupby(['ordem', 'linha']).tail(2).reset_index(drop=True)

counts = df_last_two.groupby(['ordem', 'linha']).size()
to_duplicate = counts[counts == 1].index

duplicated_rows = df_last_two.set_index(['ordem', 'linha']).loc[to_duplicate].reset_index()
df_last_two = pd.concat([df_last_two, duplicated_rows]).sort_values(['ordem', 'linha'])


df_last_two = df_last_two[['ordem','linha','latitude','longitude','datahoraservidor']]


## Join data to predict with last two points dataframe


In [5]:
test['linha'] = test['linha'].astype(str)
test['latitude'] = test['latitude'].str.replace(',', '.').astype(float)
test['longitude'] = test['longitude'].str.replace(',', '.').astype(float)

df_test = test[test['linha'].isin(valid_linhas)]

# Join the two dataframes
join_df = pd.merge(df_test, df_last_two, on=['ordem','linha'], how='inner', suffixes=('_test', '_last_two'))
join_df

,id,linha,ordem,latitude_test,longitude_test,latitude_last_two,longitude_last_two,datahoraservidor
0,835090919,867,D86049,-22.98619,-43.59132,-23.03359,-43.56296,1715867969000
1,835090919,867,D86049,-22.98619,-43.59132,-23.03119,-43.56398,1715868000000
2,1621540253,108,A41351,-22.94945,-43.18871,-22.89900,-43.21208,1715867961000
3,1621540253,108,A41351,-22.94945,-43.18871,-22.89899,-43.21209,1715867992000
4,4239337617,638,B44616,-22.85432,-43.37753,-22.86795,-43.35186,1715867964000
...,...,...,...,...,...,...,...,...
323623,999967629119048,371,C51576,-22.87720,-43.34015,-22.88422,-43.29759,1715867978000
323624,999977830770220,371,C51584,-22.91067,-43.20764,-22.90287,-43.24120,1715867978000
323625,999977830770220,371,C51584,-22.91067,-43.20764,-22.90561,-43.23996,1715868009000
323626,999985479468595,232,B25530,-22.90870,-43.17025,-22.90441,-43.18331,1715867964000


In [6]:
print(test.columns)

Index(['id', 'linha', 'ordem', 'latitude', 'longitude'], dtype='object')


## Model - Predict the next position of the bus based on past positions

In [7]:
database_url = os.getenv("DATABASE_URL")
engine = create_engine(database_url, connect_args={'client_encoding': 'latin1'})

In [8]:
def execute_query(connection, linha, lat1, lon1, lat2, lon2, last_date, prediction_lat, prediction_lon):
    query = """
    WITH initial_similar_points AS (
        SELECT time_ranking,
               ordem,
               linha,
               x,
               y,
               datahoraservidor
        FROM vw_buses_order
        WHERE linha = :linha
        AND x = width_bucket(:lon1, -43.726090, -42.951470, 1587)
        AND y = width_bucket(:lat1, -23.170790, -22.546410, 1389)
        AND (
                (datahoraservidor >= TO_TIMESTAMP(:last_date) - interval '7 day' - interval '2 hour'  
                AND datahoraservidor < TO_TIMESTAMP(:last_date) - interval '7 day' + interval '2 hour') 
                OR 
                (datahoraservidor >= TO_TIMESTAMP(:last_date) - interval '14 day' - interval '2 hour'  
                AND datahoraservidor < TO_TIMESTAMP(:last_date) - interval '14 day' + interval '2 hour')
                OR 
                (datahoraservidor >= TO_TIMESTAMP(:last_date) - interval '21 day' - interval '2 hour'  
                AND datahoraservidor < TO_TIMESTAMP(:last_date) - interval '21 day' + interval '2 hour')
            )
        AND time_ranking > 1
        LIMIT 10
    ), anterior_points AS (
        SELECT DISTINCT ON (time_ranking, ordem, linha) 
            time_ranking,
            ordem,
            linha,
            x,
            y,
            datahoraservidor
        FROM vw_buses_order
        WHERE (ordem, linha, time_ranking) IN (
            SELECT ordem, linha, time_ranking - 1
            FROM initial_similar_points
            )
    ), direction_points AS (
         SELECT 
            sp.ordem,
            sp.datahoraservidor
        FROM initial_similar_points sp
        INNER JOIN anterior_points ap
            ON sp.ordem = ap.ordem
            AND sp.linha = ap.linha
            AND sp.time_ranking = ap.time_ranking + 1
        WHERE ((ap.x - sp.x) * (:lon2 - :lon1) + (ap.y - sp.y) * (:lat2 - :lat1)) >= 0
    ), first_future_points AS (
        SELECT 
            vo.ordem,
            vo.datahoraservidor - dp.datahoraservidor AS time_diff
        FROM (
                SELECT 
                          ordem,
                          linha,
                          x,
                          y,
                          datahoraservidor
                FROM vw_buses_order
                WHERE linha = :linha
                AND ordem IN (SELECT DISTINCT ordem FROM direction_points)
             ) vo
        INNER JOIN direction_points dp
            ON vo.ordem = dp.ordem
            AND vo.datahoraservidor > dp.datahoraservidor
            AND vo.datahoraservidor < dp.datahoraservidor + interval '1 hour' + interval '20 minutes'
        WHERE vo.x < width_bucket(:prediction_lon, -43.726090, -42.951470, 1587) + 2
        AND vo.x > width_bucket(:prediction_lon, -43.726090, -42.951470, 1587) - 2
        AND vo.y < width_bucket(:prediction_lat, -23.170790, -22.546410, 1389) + 2
        AND vo.y > width_bucket(:prediction_lat, -23.170790, -22.546410, 1389) - 2
    ), selected_future_points AS (
        SELECT time_diff
        FROM first_future_points
    )
    SELECT 
        percentile_cont(0.5) WITHIN GROUP (ORDER BY time_diff) AS median_time_diff
    FROM selected_future_points;
    """
    
    params = {
        'linha': linha,
        'lat1': lat1,
        'lon1': lon1,
        'lat2': lat2,
        'lon2': lon2,
        'last_date': str(last_date),
        'prediction_lat': prediction_lat,
        'prediction_lon': prediction_lon
    }
    
    
    result = connection.execute(text(query), params)
    row = result.fetchone()
        
    return row[0]

In [9]:
join_df

,id,linha,ordem,latitude_test,longitude_test,latitude_last_two,longitude_last_two,datahoraservidor
0,835090919,867,D86049,-22.98619,-43.59132,-23.03359,-43.56296,1715867969000
1,835090919,867,D86049,-22.98619,-43.59132,-23.03119,-43.56398,1715868000000
2,1621540253,108,A41351,-22.94945,-43.18871,-22.89900,-43.21208,1715867961000
3,1621540253,108,A41351,-22.94945,-43.18871,-22.89899,-43.21209,1715867992000
4,4239337617,638,B44616,-22.85432,-43.37753,-22.86795,-43.35186,1715867964000
...,...,...,...,...,...,...,...,...
323623,999967629119048,371,C51576,-22.87720,-43.34015,-22.88422,-43.29759,1715867978000
323624,999977830770220,371,C51584,-22.91067,-43.20764,-22.90287,-43.24120,1715867978000
323625,999977830770220,371,C51584,-22.91067,-43.20764,-22.90561,-43.23996,1715868009000
323626,999985479468595,232,B25530,-22.90870,-43.17025,-22.90441,-43.18331,1715867964000


In [10]:
median_time_diff_list = []

with engine.connect() as connection:
    for i in range(0, len(join_df)- 1, 2):
        row1 = join_df.iloc[i + 1]
        row2 = join_df.iloc[i]
        median_time_diff = execute_query(
            connection,
            row1['linha'], 
            float(row1['latitude_last_two']), 
            float(row1['longitude_last_two']), 
            float(row2['latitude_last_two']), 
            float(row2['longitude_last_two']), 
            row1['datahoraservidor']/1000, # Convert to seconds - Last Date
            float(row1['latitude_test']),
            float(row1['longitude_test'])
        )

        median_time_diff_list.extend([median_time_diff, median_time_diff])

join_df['median_time_diff'] = median_time_diff_list

df_prediction = join_df[['id','datahoraservidor', 'median_time_diff']]

### Count nulls (%)

In [11]:
(df_prediction.isnull().sum()/len(df_prediction)) * 100

id                    0.0
datahoraservidor      0.0
median_time_diff    100.0
dtype: float64

### Sum interval to see the real time of the bus

In [12]:
# Sum datahoraservidor and median_time_diff
df_prediction['median_time_diff'] = pd.to_timedelta(df_prediction['median_time_diff'])
df_prediction['real_time'] =  df_prediction['datahoraservidor'] + df_prediction['median_time_diff'].dt.total_seconds() * 1000


C:\Users\Lucas S. Tavares\AppData\Local\Temp\ipykernel_8672\1776396432.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_prediction['median_time_diff'] = pd.to_timedelta(df_prediction['median_time_diff'])
C:\Users\Lucas S. Tavares\AppData\Local\Temp\ipykernel_8672\1776396432.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_prediction['real_time'] =  df_prediction['datahoraservidor'] + df_prediction['median_time_diff'].dt.total_seconds() * 1000


### Fill Na values with starting prediction hour + 30 minutes


In [13]:
# Keep only odd indices
df_prediction = df_prediction.iloc[1::2]

In [14]:
df_prediction['real_time'] = df_prediction['real_time'].fillna(df_prediction['datahoraservidor'].max() + 30 * 60 * 1000)
df_prediction['real_time'] = df_prediction['real_time'].astype('Int64')

In [15]:
df_prediction = df_prediction[['id','real_time']]

In [16]:
df_prediction.drop_duplicates(inplace=True)

## Criar json com resposta

In [17]:
test_files[0][25:36]

'2024-05-16_'

In [18]:
df_prediction['id'] = df_prediction['id'].astype('Int64')
previsoes = df_prediction.values.tolist()

match = re.search(r'teste-(\d{4}-\d{2}-\d{2})_(\d{2})', test_files[0])
date_part = match.group(1)
hour_part = match.group(2)
datahora = f"{date_part} {hour_part}:00:00"
datahora = "2024-05-16 12:00:00"

output = {
    "aluno": "Lucas Tavares Da Silva Ferreira",
    "datahora": datahora,
    "previsoes": [[str(item) if isinstance(item, pd.Int64Dtype) else item for item in row] for row in previsoes],
    "senha": "SfC-/CM5wefsN62"
}

output_json = json.dumps(output, indent=4)

output_filename = test_files[0][25:36] + "_answer.json"
with open(output_filename, "w") as json_file:
    json_file.write(output_json)

print(f"JSON salvo em {output_filename}")

JSON salvo em 2024-05-16__answer.json


### Fazer a requisição POST

In [19]:
# Fazer o POST usando a biblioteca requests
url = 'https://barra.cos.ufrj.br:443/datamining/rpc/avalia'
headers = {
    'accept': 'application/json',
    'Content-Type': 'application/json'
}

In [20]:
response = requests.post(url, headers=headers, data=output_json)

# Verificar a resposta
if response.status_code == 200:
    print("POST bem-sucedido!")
    print("Resposta do servidor:")
    print(response.json())
else:
    print(f"Falha no POST: {response.status_code}")
    print(response.text)

POST bem-sucedido!
Resposta do servidor:
{'msg': 'Problemas!', 'arquivo teste': 'teste-2024-05-16_12.json', 'rmse': 1044.2839654847498, 'ids não encontrados': 0, 'ids testados': 161814, 'total na tabela': 163210}


## Compare the distance between predicted and real values

In [21]:
answer = pd.read_json(answer_files[0], encoding='latin-1')


In [22]:
final_df = pd.merge(df_prediction, answer, on='id', how='inner', suffixes=('_pred', '_true'))

df = pd.DataFrame()

df['error'] = final_df['datahora'] - final_df['real_time']
df['error'] = df['error']/1000

# Calcular o erro médio absoluto (MAE)
mae = df['error'].abs().mean()

# Calcular a mediana do erro
median = df['error'].median()

# Calcular o erro quadrático médio (MSE)
mse = (df['error'] ** 2).mean()
rmse = np.sqrt(mse)
# Calcular o desvio padrão (STD) do erro
std = df['error'].std()

## Statistics

In [23]:
mean_error = mae
std_deviation = std
median_error = median
rmse_error = rmse


print(f'Average error in seconds: {mean_error:.2f} s')
print(f'Standard deviation of the error in seconds: {std_deviation:.2f} s')
print(f'Median error in seconds: {median_error:.2f} s')
print(f'Mean squared error in seconds: {rmse_error:.2f} s')

Average error in seconds: <NA> s
Standard deviation of the error in seconds: <NA> s
Median error in seconds: <NA> s
Mean squared error in seconds: <NA> s
